### 2015-1 2do Recuperatorio
Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”.
Programar en map-reduce un programa que genere como resultado un listado de tipo (patron,frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos. Usar combiners como método de agregación. 

In [2]:
textsRDD = sc.textFile('data/texts.txt')
textsRDD.take(2)

[u'Lorem ascent ipsum dolor sit amet, autem melius ea pri, pro malis theophrastus ut, in duo vide vocibus instructior. Duo denique torquatos dissentiunt ei, cu reprimique interesset mei. Duo nulla paulo ea. Ex commodo convenire gubergren mea, verterem lobortis ut mea. Qui debet ocurreret hendrerit ea, iracundia rationibus cotidieque et sea. recontrarepetida recontrarepetida recontrarepetida recontrarepetida',
 u'Ius secant postmaster nibh utinam ut. Vel stamp store mollis integre labores an. Qualisque incorrupte mea ut, labitur sententiae theophrastus est ea. Usu in alterum rationibus, vix dico prodesset ex, suas offendit postulant  recontrarepetida recontrarepetida recontrarepetida ne eum. Has causae corrumpit te.']

In [ ]:
def pattern(word):
    #TODO

In [ ]:
textsRDD.flatMap(lambda x: x.split())\
        .map(lambda x: (pattern(x), 1))\
        .reduceByKey(lambda a, b: a + b)\
        .takeOrdered(10, lambda x: -x[1])

---
### 2015-1 1er Recuperatorio
Se tiene una colección de documentos (textos) y se quiere calcular el promedio de la cantidad de caracteres que tienen las frases de 2,3 y 4 palabras en toda la colección. Programar la solución a este problema usando Map-Reduce usando combiners como método de agregación. 

In [3]:
textsRDD.take(2)

[u'Lorem ascent ipsum dolor sit amet, autem melius ea pri, pro malis theophrastus ut, in duo vide vocibus instructior. Duo denique torquatos dissentiunt ei, cu reprimique interesset mei. Duo nulla paulo ea. Ex commodo convenire gubergren mea, verterem lobortis ut mea. Qui debet ocurreret hendrerit ea, iracundia rationibus cotidieque et sea. recontrarepetida recontrarepetida recontrarepetida recontrarepetida',
 u'Ius secant postmaster nibh utinam ut. Vel stamp store mollis integre labores an. Qualisque incorrupte mea ut, labitur sententiae theophrastus est ea. Usu in alterum rationibus, vix dico prodesset ex, suas offendit postulant  recontrarepetida recontrarepetida recontrarepetida ne eum. Has causae corrumpit te.']

In [18]:
def kphrases(k, s):
    s = s.split()
    return [ ' '.join(s[i:i+k]) for i in xrange(len(s) - k + 1) ]

s = "use the force"
print(kphrases(2,s))

['use the', 'the force']


In [24]:
# Promedio de la cantidad de caracteres que tienen las frases de 2, 3 y 4 palabras
# A partir de tuplas (cant_palabras_de_frase, (cant_letras, 1))
# Obtiene (cant_palabras_de_frase, promedio de caracteres)
textsRDD.flatMap(lambda x: kphrases(2, x) + kphrases(3, x) + kphrases(4, x))\
        .map(lambda x: (len(x.split()), (len(x), 1)))\
        .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))\
        .map(lambda x: (x[0], x[1][0]/x[1][1]))\
        .collect()

[(2, 13), (4, 27), (3, 20)]

---
### 2014-1 2do Recuperatorio
Se tiene una colección de textos almacenados en un cluster. Se quiere construir un índice invertido para la colección completa. Programar usando Map-Reduce la construcción del índice usando el esquema TF-IDF. 

In [33]:
textsRDD = sc.textFile('data/textsWoPunctuation.txt')
textsRDD.take(10)

[u'casa casa arbol',
 u'alicia arbol casa',
 u'arbol arbol paisaje alicia',
 u'casa chalet']

In [34]:
N = textsRDD.count()
print(N)

4


In [38]:
# Para asignar un id a cada documento sin iterar entre todos, utilizo su hash
# Guardar el documento entero es una alternativa derrochadora de espacio, no? :D
def tf(doc):
    terms = doc.split()
    tfs = []
    for term in terms:
        tfs.append((term, (hash(doc), doc.count(term))))
    return tfs

In [36]:
from math import log10

def idf(term_data, N):
    return ((term_data[0], log10(N + 1 / len(term_data[1]))), term_data[1])

In [39]:
# Registros con formato ((termino, idf), [(documento, tf), ...])
textsRDD.flatMap(lambda x: tf(x)).distinct().groupByKey().mapValues(list)\
        .map(lambda x: idf(x, N)).collect()

[((u'paisaje', 0.6989700043360189), [(-5201680601302070328, 1)]),
 ((u'casa', 0.6020599913279624),
  [(262993394597099470, 1),
   (-7584191961489395017, 2),
   (6988209688724798962, 1)]),
 ((u'alicia', 0.6020599913279624),
  [(-5201680601302070328, 1), (6988209688724798962, 1)]),
 ((u'chalet', 0.6989700043360189), [(262993394597099470, 1)]),
 ((u'arbol', 0.6020599913279624),
  [(6988209688724798962, 1),
   (-7584191961489395017, 1),
   (-5201680601302070328, 2)])]